In [1]:
import polars as pl
import pyodbc
from datetime import date

import time
start_time = time.time()

In [2]:
from IPython import get_ipython
get_ipython().run_line_magic("run", "functions.ipynb")

#### Definición de Rutas

In [3]:
cierre = "202502"
cierre_l = "Feb25"

# Carpetas
path = "E:/Users/jhernandezr/DAR/garantias/reporte/fotos/"
wd_data_raw = path + "data/raw/"
wd_data_external = path + "data/external/"

# Archivos
fl_desembolsos_p1 = wd_data_raw + f'Desembolsos_P1_{cierre}'
fl_desembolsos_p1_bmxt = wd_data_raw + f'Desembolsos_P1_80686_{cierre}'
fl_desembolsos_p1_fianzas = wd_data_raw + f'Desembolsos_P1_BMXT_{cierre}'
fl_desembolsos_p2 = wd_data_raw + f'Desembolsos_P2_{cierre}'
fl_desembolsos_p2_bmxt = wd_data_raw + f'Desembolsos_P2_80686_{cierre}'
fl_desembolsos_p2_fianzas = wd_data_raw + f'Desembolsos_P2_BMXT_{cierre}'
fl_catalogos = wd_data_external + f'Catálogos_{cierre_l}'



### Catálogos

In [10]:
conn_str = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + fl_catalogos + '.accdb'
conn = pyodbc.connect(conn_str)

# Solo Recuperaciones
estatus = pl.read_database(query='SELECT * FROM [ESTATUS]', connection=conn)

# Recuperaciones, Pagos
tipo_cambio = pl.read_database(query='SELECT * FROM [TIPO CAMBIO]', connection=conn)
programa = pl.read_database(query='SELECT * FROM [PROGRAMA]', connection=conn, schema_overrides={'PROGRAMA_ID': pl.Int128})
agrupamiento = pl.read_database(query='SELECT * FROM [AGRUPAMIENTO]', connection=conn)
udis = pl.read_database(query='SELECT * FROM [UDIS]', connection=conn, schema_overrides={'Fecha_Paridad': pl.Datetime})
#udis = udis.with_columns(pl.col("Fecha_Paridad").cast(pl.Date))
tipo_credito = pl.read_database(query='SELECT * FROM [TIPO_CREDITO]', connection=conn, schema_overrides={'Tipo_Credito_ID': pl.Int32})
tipo_garantia = pl.read_database(query='SELECT * FROM [TIPO_GARANTIA]', connection=conn, schema_overrides={'Tipo_garantia_ID': pl.Int32})
sfc = pl.read_database(query='SELECT * FROM [SIN FONDOS CONTRAGARANTIA]', connection=conn)

conn.close()

tipo_cambio.columns = ["Anio", "Mes", "TC"]
cierre = date(2024,12,31)
tdc = tipo_cambio.filter((pl.col("Anio")==cierre.year) & (pl.col("Mes")==cierre.month)).select("TC").item()
tdc

20.8829

### Querie Pagos

In [5]:
pagos_f1 = import_f1()
pagos_f2 = import_f2()

In [11]:
pagos_f1 = genera_concatenado(pagos_f1)
pagos_f1 = genera_tpro_clave(pagos_f1)
pagos_f2 = genera_concatenado(pagos_f2)
pagadas_global_inter = genera_pagadas_global_inter(pagos_f1, pagos_f2)

In [9]:
udis

Fecha_Paridad,Paridad_Peso
date,f64
2024-05-20,8.149012
2024-05-21,8.150367
2024-05-22,8.151722
2024-05-23,8.153078
2024-05-24,8.154433
…,…
2025-01-27,8.3685
2025-01-28,8.36956
2025-01-29,8.37062


In [12]:

pagadas_global_vf = genera_pagadas_global_vf(pagadas_global_inter)


In [13]:
def main_querie_pagadas():
    pagos_f1 = import_f1()
    pagos_f2 = import_f2()

    # Crea Concatenados y TPRO_CLAVE
    pagos_f1 = genera_concatenado(pagos_f1)
    pagos_f1 = genera_tpro_clave(pagos_f1)
    pagos_f2 = genera_concatenado(pagos_f2)

    pagadas_global_inter = genera_pagadas_global_inter(pagos_f1, pagos_f2)
    pagadas_global_vf = genera_pagadas_global_vf(pagadas_global_inter)
    pagadas_global_vf = complementa_pagadas_global_vf(pagadas_global_vf)

    valida_pagos = genera_validador(pagos_f2)
    pagadas_global_vf_bancomext = genera_pagadas_global_bancomext(pagadas_global_vf)
    pagadas_global_vf_sin_bancomext = genera_pagadas_global_sin_bancomext(pagadas_global_vf)

    valida_base_pagos = genera_valida_base_pagos(pagadas_global_vf)
    valida_base_pagos_mn = genera_valida_base_pagos_mn(pagadas_global_vf)

    pagadas_global_vf.write_csv('./../data/processed/DWH/pagadas_global_vf.csv')
    pagadas_global_vf_bancomext.write_csv('./../data/processed/DWH/pagadas_global_vf_bancomext.csv')
    pagadas_global_vf_sin_bancomext.write_csv('./../data/processed/DWH/pagadas_global_vf_sin_bancomext.csv')
    valida_pagos.write_csv('./../data/processed/DWH/valida_pagos.csv')
    valida_base_pagos.write_csv('./../data/processed/DWH/valida_base_pagos.csv')
    valida_base_pagos_mn.write_csv('./../data/processed/DWH/valida_base_pagos_mn.csv')

    return pagadas_global_vf

pagadas_global_vf = main_querie_pagadas()

### Querie Recuperaciones

#### Funciones

In [9]:
schema_recuperaciones = {
    'ANIO DEGL': pl.String,
    'DESC_INDICADOR': pl.String,
    'DESCRIPCION': pl.String,
    'ESTATUS': pl.String,
    'FECHA': pl.Date,
    'FECHA_APERTURA': pl.Date,
    'FECHA_CONSULTA': pl.Date,
    'FECHA_GARANTIA_HONRADA': pl.Date,
    'FECHA_REGISTRO': pl.Date,
    'FECHA_REGISTRO_ALTA': pl.Date,
    'FISO_ID': pl.Int32,
    'HISTORICO': pl.String,
    'ID': pl.Int64,
    'INTERMEDIARIO_ID': pl.String,
    'MES DEGL': pl.Int8,
    'MONEDA_ID': pl.Int8,
    'NOMBRE_EMPRESA': pl.String,
    'NUMERO_CREDITO': pl.String,
    'PORCENTAJE_GARANTIZADO': pl.Float32,
    'PROGRAMA_ID': pl.Int32,
    'PROGRAMA_ORIGINAL': pl.Int32,
    'RAZON_SOCIAL': pl.String,
    'RFC_EMPRESA': pl.String,
    'TIPO_CAMBIO_GARANTIA': pl.Float32,
    'TIPO_CREDITO_ID': pl.Int32,
    'TIPO_GARANTIA_ID': pl.Int32,
    'TIPO_PERSONA': pl.String,
    'GASTO_JUICIOS': pl.Float32,
    'INTER_MORAT':pl.Float32,
    'INTERES_GENERADO':pl.Float32,
    'INTERESES': pl.Float32,
    'MONTO': pl.Float32,
    'MONTO_CREDITO_MN': pl.Float64,
    'MORATORIOS': pl.Float32,
    'Número de registros': pl.Int8,
    'PENALIZACION': pl.Float32
}


schema_recuperaciones_bancomext = {
    'DESC_INDICADOR': pl.String,
    'DESCRIPCION': pl.String,
    'ESTATUS': pl.String,
    'FECHA': pl.Date,
    'FECHA_APERTURA': pl.Date,
    'FECHA_CONSULTA': pl.Date,
    'FECHA_GARANTIA_HONRADA': pl.Date,
    'FECHA_REGISTRO': pl.Date,
    'FECHA_REGISTRO_ALTA': pl.Date,
    'HISTORICO': pl.String,
    'ID': pl.Int64,
    'INTERMEDIARIO_ID': pl.String,
    'MONEDA_ID': pl.Int8,
    'NOMBRE_EMPRESA': pl.String,
    'NUMERO_CREDITO': pl.String,
    'PORCENTAJE_GARANTIZADO': pl.Float32,
    'PROGRAMA_ID': pl.Int32,
    'PROGRAMA_ORIGINAL': pl.Int32,
    'RAZON_SOCIAL': pl.String,
    'RFC_EMPRESA': pl.String,
    'TIPO_CAMBIO_GARANTIA': pl.Float32,
    'TIPO_CREDITO_ID': pl.Int32,
    'TIPO_GARANTIA_ID': pl.Int32,
    'TIPO_PERSONA': pl.String,
    'GASTO_JUICIOS': pl.String,
    'INTER_MORAT': pl.Float32,
    'INTERES_GENERADO': pl.Float32,
    'INTERESES': pl.Float32,
    'MONTO': pl.Float32,
    'MONTO_CREDITO_MN': pl.Float64,
    'MORATORIOS': pl.Float32,
    'Número de registros': pl.Int8,
    'PENALIZACION': pl.Float32
}

In [10]:
def importa_recuperaciones():
    dwh_recup = pl.read_csv('../data/raw/SRC_FL_RECUP_MES_ANT_Datos migrados.csv', schema=schema_recuperaciones)
    dwh_recup = dwh_recup.drop(['ANIO DEGL', 'FISO_ID', 'MES DEGL'])

    dwh_recup_bmxt = pl.read_csv('../data/raw/SRC_FLUJOS_RECUPERACIONES_BANCOMEXT_Datos migrados.csv', schema=schema_recuperaciones_bancomext)
    dwh_recup_bmxt = dwh_recup_bmxt.with_columns(
        pl.col("GASTO_JUICIOS").str.replace(",", "").cast(pl.Float32)
    )

    aux = pl.concat([dwh_recup, dwh_recup_bmxt], rechunk=True)

    dwh_recuperaciones = aux.rename({
        "DESC_INDICADOR": "Producto",
        "FECHA_REGISTRO_ALTA": "Fecha Registro Alta",
        "NOMBRE_EMPRESA": "Empresa / Acreditado (Descripción)",
        "RAZON_SOCIAL": "Razón Social (Intermediario)",
        "RFC_EMPRESA": "RFC Empresa / Acreditado",
        "TIPO_CAMBIO_GARANTIA":"Tipo_Cambio_Cierre",
        "GASTO_JUICIOS": "Gastos Juicio",
        "INTER_MORAT": "Moratorios",
        "INTERES_GENERADO": "Interes Generado", 
        "INTERESES": "Interes", 
        "MONTO": "Monto",
        "MONTO_CREDITO_MN": "Monto _Credito_Mn",
        "MORATORIOS": "Excedente",
        "PENALIZACION": "Penalizacion"
    }).select([
        "Producto", 
        "DESCRIPCION", 
        "ESTATUS", 
        "FECHA", 
        "FECHA_APERTURA", 
        "FECHA_CONSULTA", 
        "FECHA_GARANTIA_HONRADA", 
        "FECHA_REGISTRO",
        "Fecha Registro Alta",
        "HISTORICO", 
        "ID", 
        "INTERMEDIARIO_ID", 
        "MONEDA_ID", 
        "Empresa / Acreditado (Descripción)", 
        "NUMERO_CREDITO", 
        "PORCENTAJE_GARANTIZADO", 
        "PROGRAMA_ID", 
        "PROGRAMA_ORIGINAL", 
        "Razón Social (Intermediario)", 
        "RFC Empresa / Acreditado",
        "Tipo_Cambio_Cierre", 
        "TIPO_CREDITO_ID", 
        "TIPO_GARANTIA_ID", 
        "TIPO_PERSONA", 
        "Gastos Juicio", 
        "Moratorios", 
        "Interes Generado", 
        "Interes", 
        "Monto", 
        "Monto _Credito_Mn", 
        "Excedente",
        "Penalizacion"
    ])

    dwh_recuperaciones = dwh_recuperaciones.with_columns(pl.col("TIPO_GARANTIA_ID").fill_null(999))
    dwh_recuperaciones = genera_tpro_clave(dwh_recuperaciones)

    return dwh_recuperaciones

In [11]:
def genera_recuperadas_global_inter(df):
    # Requiere que se hayan importado los catálogos
    result = (df
    .join(programa.select(['PROGRAMA_ID', 'AGRUPAMIENTO_ID', 'ESQUEMA', 'SUBESQUEMA']), on="PROGRAMA_ID", how='left')
        .join(agrupamiento, on='AGRUPAMIENTO_ID', how='left')
        .join(udis, left_on="FECHA_APERTURA", right_on="Fecha_Paridad", how='left')
        .join(tipo_credito.select(['Tipo_Credito_ID', 'NR_R']), left_on='TIPO_CREDITO_ID', right_on="Tipo_Credito_ID", how='left')
        .join(tipo_garantia.select(['Tipo_garantia_ID', 'CSG']), left_on='TIPO_GARANTIA_ID', right_on='Tipo_garantia_ID', how='left')
        .join(sfc.select(['Intermediario_Id', 'CLAVE_CREDITO', 'FONDOS_CONTRAGARANTIA']), left_on=['INTERMEDIARIO_ID', 'NUMERO_CREDITO'], right_on=['Intermediario_Id', 'CLAVE_CREDITO'], how='left')
        .join(estatus.select(['Estatus ID', 'Recup/Rescat']), left_on='ESTATUS', right_on='Estatus ID', how='left')
        ).rename({'Recup/Rescat': 'Recup_Rescat'})

    # Complementa
    result = (result
        .with_columns(pl.when(pl.col('MONEDA_ID')==54)
                    .then(tdc).otherwise(1).alias("TC"))
        .with_columns(pl.when(pl.col("Monto _Credito_Mn")<=(900000*pl.col("Paridad_Peso")))
                    .then(0).otherwise(1).alias("MM_UDIS"))
        .with_columns(pl.when(pl.col("FONDOS_CONTRAGARANTIA")=="SF")
                    .then(pl.lit("SF")).otherwise(pl.lit("CF")).alias("CSF"))
        )
    
    return result

In [12]:
def genera_recuperadas_global_vf(df):
    aux = df.select(['Monto', 'Interes', 'Moratorios', 'Excedente']).sum_horizontal(ignore_nulls=True)
    result = (df
        .with_columns((pl.col("Monto") * pl.col("TC")).alias("Monto_Mn"))
        .with_columns((pl.col("Interes") * pl.col("TC")).alias("Interes_Mn"))
        .with_columns((pl.col("Moratorios") * pl.col("TC")).alias("Moratorios_Mn"))
        .with_columns((pl.col("Excedente") * pl.col("TC")).alias("Excedente_Mn"))
        .with_columns((pl.col("Gastos Juicio") * pl.col("TC")).alias("Gastos_Juicio_Mn"))
        .with_columns((aux * pl.col("TC")).alias("Sub_Total_Mn"))
        .with_columns((pl.col("Sub_Total_Mn")-pl.col("Gastos_Juicio_Mn")).alias("Monto_Total_Mn"))
        .with_columns(pl.when(((pl.col("NUMERO_CREDITO")=="9842725312") & (pl.col("INTERMEDIARIO_ID")=="10040012")))
               .then(date(2011,8,2))
               .otherwise(pl.col("FECHA_GARANTIA_HONRADA"))
               .alias("FECHA_GARANTIA_HONRADA"))
    )
    return result

In [13]:
def genera_recuperadas_global_bancomext(df):
    result = (df.filter(
        (pl.col("Producto") == "GARANTIAS BANCOMEXT") |
        (pl.col("Producto") == "GARANTIAS SHF/LI FINANCIERO") |
        (pl.col("Producto") == "GARANTIAS BANSEFI") |
        (pl.col("Producto").is_null())
    ))
    return result

def genera_recuperadas_global_sin_bancomext(df):
    result = (df.filter(
        (pl.col("Producto") != "GARANTIAS BANCOMEXT") |
        (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") |
        (pl.col("Producto") != "GARANTIAS BANSEFI")
    ))
    return result

In [14]:
def genera_valida_dwh_dac(df):
    result = (df
      .group_by(['HISTORICO', 'Tipo_Cambio_Cierre', 'Producto'])
      .agg(pl.col("Monto").sum().alias("S_Monto"),
            pl.col("Interes").sum().alias("S_Interes"),
            pl.col("Moratorios").sum().alias("S_Moratorios"),
            pl.col("Excedente").sum().alias("S_Excedente"),
            pl.col("Gastos Juicio").sum().alias("S_Gastos_Juicio"),
          )
    )
    return result

In [15]:
def genera_valida_td(df):
    result = (df
      .group_by(['Producto', 'Tipo_Cambio_Cierre'])
      .agg(pl.col("Monto").sum().alias("S_Monto"),
            pl.col("Interes").sum().alias("S_Interes"),
            pl.col("Moratorios").sum().alias("S_Moratorios"),
            pl.col("Excedente").sum().alias("S_Excedente"),
            pl.col("Gastos Juicio").sum().alias("S_Gastos_Juicio"),
          )
      )
    return result

#### Ejecución

In [16]:
def main_querie_recuperaciones():
    dwh_recuperaciones = importa_recuperaciones()
    recuperadas_global_inter = genera_recuperadas_global_inter(dwh_recuperaciones)
    recuperadas_global_vf = genera_recuperadas_global_vf(recuperadas_global_inter)

    recuperadas_global_vf_bancomext = genera_recuperadas_global_bancomext(recuperadas_global_vf)
    recuperadas_global_vf_sin_bancomext = genera_recuperadas_global_sin_bancomext(recuperadas_global_vf)

    valida_dwh_dac = genera_valida_dwh_dac(dwh_recuperaciones)
    valida_td = genera_valida_td(dwh_recuperaciones)

    recuperadas_global_vf.write_csv('./../data/processed/DWH/python/recuperadas_global_vf.csv')
    recuperadas_global_vf_bancomext.write_csv('./../data/processed/DWH/python/recuperadas_global_vf_bancomext.csv')
    recuperadas_global_vf_sin_bancomext.write_csv('./../data/processed/DWH/python/recuperadas_global_vf_sin_bancomext.csv')
    valida_dwh_dac.write_csv('./../data/processed/DWH/python/valida_dwh_dac.csv')
    valida_td.write_csv('./../data/processed/DWH/python/valida_td.csv')

    return recuperadas_global_vf

In [17]:
#main_querie_recuperaciones()

### Querie Union Flujos

#### Funciones

In [18]:
def genera_pagos_agrup(pagadas_global_vf):
    pagos_agrup = pagadas_global_vf.group_by([
        'Producto', 
        'INTERMEDIARIO_ID', 
        'NUMERO_CREDITO', 
        'FECHA_GARANTIA_HONRADA', 
        'MM_UDIS', 
        'NR_R', 
        'Razón Social (Intermediario)', 
        'TPRO_CLAVE', 
        'AGRUPAMIENTO'
    ]).agg(
        pl.col("Monto_Desembolso_Mn").sum().alias("Monto_Desem_Mn"),
        pl.col("Interes_Desembolso_Mn").sum().alias("Interes_Desem_Mn"),
        pl.col("Interes_Moratorios_Mn").sum().alias("Interes_Morat_Mn"),
        pl.col("Monto_Pagado_Mn").sum().alias("MPagado_Mn"),
    )


    return pagos_agrup

In [19]:
def genera_recuperaciones_agrup(recuperadas_global_vf):
    recuperaciones_agrup = (recuperadas_global_vf
        .group_by(["ESTATUS", "INTERMEDIARIO_ID", "NUMERO_CREDITO", "Concatenado"])
        .agg(pl.col("Monto_Mn").sum().alias("Monto_Recup_Mn"),
            pl.col("Interes_Mn").sum().alias("Interes_Recup_Mn"),
            pl.col("Moratorios_Mn").sum().alias("Moratorios_Recup_Mn"),
            pl.col("Excedente_Mn").sum().alias("Excedente_Recup_Mn"),
            pl.col("Monto_Total_Mn").sum().alias("Monto_Total_Recup_Mn"))
        )
    return recuperaciones_agrup


In [20]:
def genera_recupera_con_pagos_flujos(uf_recuperaciones_pagos):
    order_columns = [
        "Concatenado", 
        "FECHA_CONSULTA", 
        "PROGRAMA_ID",
        "TIPO_GARANTIA_ID", 
        "Tipo_Cambio_Cierre", 
        "PROGRAMA_ORIGINAL", 
        "PORCENTAJE_GARANTIZADO",
        "Monto _Credito_Mn", 
        "FECHA_APERTURA", 
        "MONEDA_ID", 
        "TIPO_CREDITO_ID", 
        "FECHA_GARANTIA_HONRADA", 
        "TPRO_CLAVE",
        "NR_R", 
        "Producto", 
        "AGRUPAMIENTO_ID", 
        "AGRUPAMIENTO", 
        "INTERMEDIARIO_ID",
        "NUMERO_CREDITO", 
        "ID", 
        "Monto", 
        "Interes", 
        "Moratorios", 
        "DESCRIPCION", 
        "ESTATUS", 
        "FECHA_REGISTRO", 
        "FECHA", 
        "Monto_Mn",
        "Interes_Mn", 
        "Excedente", 
        "Excedente_Mn", 
        "Moratorios_Mn", 
        "Sub_Total_Mn",
        "Razón Social (Intermediario)",
        "Empresa / Acreditado (Descripción)",
        "RFC Empresa / Acreditado", 
        "TIPO_PERSONA", 
        "Recup_Rescat",
        "MM_UDIS", 
        "ESQUEMA", 
        "CSG", 
        "CSF", 
        "Gastos_Juicio_Mn", 
        "Monto_Total_Mn", 
        "MPagado_Mn",
        "HISTORICO", 
        "Fecha Registro Alta"
        ]

    recupera_con_pagos_flujos = uf_recuperaciones_pagos.select(order_columns)
    return recupera_con_pagos_flujos

In [21]:
def genera_recupera_con_pagos_flujos_bancomext(df):
    result = (df.filter(
        (pl.col("Producto") == "GARANTIAS BANCOMEXT") |
        (pl.col("Producto") == "GARANTIAS SHF/LI FINANCIERO") |
        (pl.col("Producto") == "GARANTIAS BANSEFI") |
        (pl.col("Producto").is_null())
    ))
    return result


def genera_recupera_con_pagos_flujos_sin_bancomext(df):
    result = (df.filter(
        (pl.col("Producto") != "GARANTIAS BANCOMEXT") |
        (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") |
        (pl.col("Producto") != "GARANTIAS BANSEFI")
    ))
    return result

def genera_valida_base_pagos_recup_mn(recupera_con_pagos_flujos):
     valida_base_pagos_mn = (recupera_con_pagos_flujos
     .group_by('Producto')
     .agg(pl.col('Monto_Mn').sum().alias('Monto_Mn_Suma'),
          pl.col('Interes_Mn').sum().alias('Interes_Mn_Suma'),
          pl.col('Moratorios_Mn').sum().alias('Moratorios_Mn_Suma'),
          pl.col('Excedente_Mn').sum().alias('Excedente_Mn_Suma'),
          pl.col('Gastos_Juicio_Mn').sum().alias('Gastos_Juicio_Mn_Suma')
          )
     )
     return valida_base_pagos_mn

def genera_valida_base_pagos_recup(recupera_con_pagos_flujos):
     valida_base_pagos = (recupera_con_pagos_flujos
     .group_by('Producto')
     .agg(pl.col('Monto').sum().alias('Monto_Suma'),
          pl.col('Interes').sum().alias('Interes_Suma'),
          pl.col('Moratorios').sum().alias('Moratorios_Suma'),
          pl.col('Excedente').sum().alias('Excedente_Suma'),
          pl.col('Gastos Juicio').sum().alias('Gastos_Juicio_Suma')
          )
     )
     return valida_base_pagos

#### Ejecución

In [22]:
def main_querie_union_flujos(pagadas_global_vf, recuperadas_global_vf):
    pagos_agrup = genera_pagos_agrup(pagadas_global_vf)
    pagos_agrup = genera_concatenado(pagos_agrup)

    recuperadas_global_vf = genera_concatenado(recuperadas_global_vf)
    recuperaciones_agrup = genera_recuperaciones_agrup(recuperadas_global_vf)

    # Falta depurar columnas de B
    uf_pagos_recuperaciones = pagos_agrup.join(recuperadas_global_vf, on="Concatenado", how='left')

    # Falta depurar columnas de B
    uf_recuperaciones_pagos = recuperadas_global_vf.join(pagos_agrup, on="Concatenado", how='left')

    t_roberto = pagos_agrup.join(recuperaciones_agrup, on="Concatenado", how='left')

    recupera_con_pagos_flujos = genera_recupera_con_pagos_flujos(uf_recuperaciones_pagos)
    recupera_con_pagos_flujos_bmxt = genera_recupera_con_pagos_flujos_bancomext(recupera_con_pagos_flujos)
    recupera_con_pagos_flujos_sin_bmxt = genera_recupera_con_pagos_flujos_sin_bancomext(recupera_con_pagos_flujos)

    valida_base_pagos_recup = genera_valida_base_pagos_recup(recuperadas_global_vf)
    valida_base_pagos_recup_mn = genera_valida_base_pagos_recup_mn(recuperadas_global_vf)
    
    recupera_con_pagos_flujos.write_csv('./../data/processed/DWH/python/recupera_con_pagos_flujos.csv')
    recupera_con_pagos_flujos_bmxt.write_csv('./../data/processed/DWH/python/recupera_con_pagos_flujos_bmxt.csv')
    recupera_con_pagos_flujos_sin_bmxt.write_csv('./../data/processed/DWH/python/recupera_con_pagos_flujos_sin_bmxt.csv')
    valida_base_pagos_recup.write_csv('./../data/processed/DWH/python/valida_base_pagos_recup.csv')
    valida_base_pagos_recup_mn.write_csv('./../data/processed/DWH/python/valida_base_pagos_recup_mn.csv')

    return 0


### Ejecución Principal

In [23]:
if __name__ ==  "__main__":
    pagadas_global_vf = main_querie_pagadas()
    recuperadas_global_vf = main_querie_recuperaciones()
    main_querie_union_flujos(pagadas_global_vf, recuperadas_global_vf)

In [24]:
end_time = time.time()
end_time - start_time

24.845292806625366